# Trails for database

24 nov 2023



In [1]:
import pandas as pd
import numpy as np
import pulp as plp
import geopandas as gpd
import os
import requests

from shapely.geometry import Point
from shapely.ops import cascaded_union
import logging
import psycopg2
import time
from sqlalchemy import create_engine

# Functions

In [9]:
# params
logging.basicConfig(level=logging.DEBUG)
FLASK_ENV = os.environ.get('FLASK_ENV')

# database connection
os.environ['POSTGRES_DB'] = 'fitpass'
os.environ['POSTGRES_USER'] = 'postgres'
os.environ['POSTGRES_HOST'] = 'localhost' # NOTA: CAMBIAR ESTO A LA HORA DE CORRER EL DOCKER
os.environ['POSTGRES_PASSWORD'] = 'skalas-puts-me-an-aplus-in-this-class'

In [10]:
def log_debugg(text):
    if FLASK_ENV == 'development':
        logging.debug(text)

def get_db_conn():
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            log_debugg(f"Trying to connect to the PostgreSQL database... ({retries}/{max_retries})")
            host = os.environ.get('POSTGRES_HOST', 'localhost')
            user = os.environ.get('POSTGRES_USER', 'postgres')
            password = os.environ.get('POSTGRES_PASSWORD', '')
            database = os.environ.get('POSTGRES_DB', 'fitpass')
            conn = create_engine(f'postgresql://{user}:{password}@{host}/{database}')
            log_debugg("Connected to the PostgreSQL database.")
            return conn
        except psycopg2.OperationalError as e:
            log_debugg(f"Error: {e}")
            log_debugg(f"Waiting 10 seconds for PostgreSQL to be ready... ({retries}/{max_retries})")
            retries += 1
            time.sleep(10)

    log_debugg("Max retries reached. Unable to connect to the PostgreSQL database.")
    return None

# Connection

## Query database

In [11]:
log_debugg("reading data")
conn = get_db_conn()
query = "select * from cdmx_studios" 
df_fitpass_r = pd.read_sql_query(query, conn)
df_fitpass_r = df_fitpass_r.drop_duplicates(subset=['gym_id']) # drop duplicates
# conn.dispose() # close connection

In [5]:
df_fitpass_r

,gym_id,gym_name,pro_status,virtual_status,class_minutes,latitude,longitude,barre,box,crossfit,...,gym,hiit,mma,pilates,pool,running,sports,virtual_class,wellness,yoga
0,3913,21159 Fit Movement,0,0,45,19.409432,-99.162129,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4356,321 Move!,0,0,None,19.359657,-99.202687,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5576,3 Are Legend,0,0,None,19.539159,-99.182145,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3735,40 Grados Hot Yoga,0,0,None,19.552389,-99.270923,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5694,60 Mind Fitness,0,0,None,19.401965,-99.155869,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,17,Zonadanza: Ballet Studio & Art,0,0,None,19.390768,-99.291066,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
862,1623,Zuda Lilas,1,0,None,19.38801,-99.247764,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
863,3976,Zuda Prado Norte,1,0,None,19.427231,-99.211101,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
864,3101,Zuda Virtual,0,1,None,19.405721,-99.252566,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


## Request API

In [6]:
# sample
sample_request = {
    "name": "roman",
    "location": {
        "latitude": 19.388900864307445,
        "longitude": -99.18265186842596
    },
    "distance_sensitivity": "medium",
    "preferences": {
        "love_activities": ["barre", "yoga", "cycling", "pilates", "gym"],
        "hate_activities": ["crossfit", "functional"]
    },
    "is_pro": 1,
    "max_allowed_classes_per_class": 4,
    "num_classes_per_month": 23
}

In [7]:
# generate post request to http://localhost:8080/predict
url = 'http://localhost:8080/predict'
r = requests.post(url, json=sample_request)
r.json()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8080
DEBUG:urllib3.connectionpool:http://localhost:8080 "POST /predict HTTP/1.1" 200 353


[{'gym_id': '5305', 'gym_times': 1},
 {'gym_id': '681', 'gym_times': 1},
 {'gym_id': '2376', 'gym_times': 1},
 {'gym_id': '1051', 'gym_times': 1},
 {'gym_id': '2621', 'gym_times': 1},
 {'gym_id': '2634', 'gym_times': 1},
 {'gym_id': '2646', 'gym_times': 1},
 {'gym_id': '5350', 'gym_times': 4},
 {'gym_id': '5410', 'gym_times': 4},
 {'gym_id': '1067', 'gym_times': 4},
 {'gym_id': '2628', 'gym_times': 4}]

In [8]:
df_final_product = pd.DataFrame(r.json())

In [ ]:
df_final_product

In [ ]:
df_work = df_final_product.merge(df_fitpass_r, on='gym_id', how='inner')
df_work